In [1]:
import pandas as pd
data = pd.read_excel("datasets/meetreq_training.xlsx")

import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

data.head()

,training,trainer,hari,waktu,peserta,dept,atasan,email_peserta,duration,meeting_room,email_meeting_room,email_trainer,email_optional,status_meetreq
0,Basic WTP,Ridi Nur Ardiansyah,Rabu,2019-08-28 14:00:00,Wildan Akbar Kombat Ginting,GEB,Sigit Aditya Kinardi,fahimhadimaula@gmail.com,120,Collaboration,rcollaborationcbt@nutrifood.co.id,prameswari.kristal@nutrifood.co.id,kristal.primbum.prima@gmail.com,NaN
1,Basic WTP,Ridi Nur Ardiansyah,Rabu,2019-08-28 14:00:00,Naufal Farras Wahono,SSA,Priarso Sukaton,fahimhadimaula@gmail.com,120,Collaboration,rcollaborationcbt@nutrifood.co.id,prameswari.kristal@nutrifood.co.id,kristal.primbum.prima@gmail.com,NaN
2,Basic WTP,Ridi Nur Ardiansyah,Rabu,2019-09-04 14:00:00,Wildan Akbar Kombat Ginting,GEB,Sigit Aditya Kinardi,fahimhadimaula@gmail.com,120,Collaboration,rcollaborationcbt@nutrifood.co.id,prameswari.kristal@nutrifood.co.id,kristal.primbum.prima@gmail.com,NaN
3,Basic WTP,Ridi Nur Ardiansyah,Rabu,2019-09-04 14:00:00,Naufal Farras Wahono,SSA,Priarso Sukaton,fahimhadimaula@gmail.com,120,Collaboration,rcollaborationcbt@nutrifood.co.id,prameswari.kristal@nutrifood.co.id,kristal.primbum.prima@gmail.com,NaN
4,Menemui Personalia Manager - Dewi Kristiani,Dewi Kristiani,Kamis,2019-08-29 14:00:00,Ade santana,GLF,Hestiana / Fajrina + Penyelia,fahimhadimaula@gmail.com,120,Wdank,rwdank@nutrifood.co.id,prameswari.kristal@nutrifood.co.id,kristal.primbum.prima@gmail.com,NaN


In [2]:
for train in data.training.unique():
    train_base = data[(data.training == train) & (data.status_meetreq.isnull())]
    for date_start in train_base.waktu.unique():
        date_base = train_base[train_base.waktu == date_start]
        foremail = date_base.loc[:, ['training', 'trainer', 'waktu', 'peserta', 'dept', 'atasan', 'email_peserta']]
        mail = outlook.CreateItem(1)
        ts = pd.to_datetime(str(date_start)) 
        d = ts.strftime('%Y-%m-%d %H:%M')
        mail.Start = d
        mail.Subject = train
        mail.Duration = date_base.duration.unique()[0].item()
        mail.Location = date_base.email_meeting_room.unique()[0]
        mail.MeetingStatus = 1
        for trainee in date_base.email_peserta.unique():
            mail.Recipients.Add(trainee)
        for trainer in date_base.email_trainer.unique():
            mail.Recipients.Add(trainer)
        for room in date_base.email_meeting_room.unique():
            booking = mail.Recipients.Add(room)
            booking.resolve
        for carboncopy in date_base.email_optional.unique():
            cc = mail.Recipients.Add(carboncopy)
            cc.Type = 2
#         body = 
        mail.Body = (r'''Dear rekan-rekan,
Mengundang rekan-rekan POK mengikuti:
POK | %(judul)s

Hari, Tanggal: %(hari)s, %(tanggal)s
Durasi: %(durasi)i jam
Tempat: %(tempat)s

%(df)s
        ''' % {"judul": train, "tanggal": d, "durasi": date_base.duration.unique()[0].item()/60, "hari": date_base.hari.unique()[0], "tempat": date_base.meeting_room.unique()[0], "df": foremail.to_html()})
        mail.Save()
        mail.Send()
        
        idx = date_base.index
        data.loc[idx, 'status_meetreq'] = 'done'
        
# hapus excel lama, terbitkan excel baru dengan update status